In [1]:
import cPickle as pkl
from cspython.scraper import modifiedSoup
from cspython.data_processing import process_scrapped, change_team_name_dataset
import cPickle as pkl
import sys
sys.setrecursionlimit(15000)
import pandas as pd
import cspython.analysis as a

In [2]:
with open('../cspython/Tempo Storm_2016-10-01_to_2017-10-13.pkl', 'rb') as f:
    d = pkl.load(f)
    
with open('../cspython/ex-paiN_2016-10-01_to_2017-10-13.pkl', 'rb') as f:
    d1 = pkl.load(f)
    
with open('../cspython/paiN_2016-10-01_to_2017-10-13.pkl', 'rb') as f:
    d2 = pkl.load(f)

In [3]:
cur_roster = ['dzt', 'land1n', 'tatazin', 'shz', 's1', 'bLecker']

In [4]:
name = 'Tempo Storm'
d1 = change_team_name_dataset(name, 'paiN', d1)
d2 = change_team_name_dataset(name, 'ex-paiN', d2)

In [104]:
with open('Tempo_combined.pkl', 'wb') as f:
    pkl.dump(d+d1+d2, f)

In [6]:
def analyze_data(d, team, cur_roster):
    ov, big_data = process_scrapped(d)
    return a.map_roster_analysis(big_data,team, cur_roster)

In [91]:
map_ratios = analyze_data(d+d1+d2, 'Tempo Storm', cur_roster)
baseline = pd.read_csv('spreadsheets/NRG.csv', index_col = 0)

In [101]:
def get_diff(baseline, cur_frame, team_name):
    diff = baseline - cur_frame
    diff = pd.concat([diff.iloc[:,:-2], cur_frame.iloc[:,-2:]], axis=1)

    multi_idx_tuples = zip([team_name]*len(diff),diff.index.tolist())
    diff.index = pd.MultiIndex.from_tuples(multi_idx_tuples)
    diff.loc[:,'sum_advantage'] = diff.loc[:, 't_win_ratio'] + diff.loc[:,'ct_win_ratio']
    diff.loc[:,'weighted_sum_advantage'] = diff.loc[:, 'roster_weighted_t_win_ratio'] + diff.loc[:,'roster_weighted_ct_win_ratio']
    diff = diff.loc[:, ['t_win_ratio',
                         'ct_win_ratio',
                         'sum_advantage',
                         'roster_weighted_t_win_ratio',
                         'roster_weighted_ct_win_ratio',
                         'weighted_sum_advantage',
                         't_rounds_played',
                         'ct_rounds_played',
                         ]]
    return diff.round(3)

In [102]:
diff = get_diff(baseline, map_ratios, 'Tempo Storm')

In [103]:
diff.to_excel('Tempo_Storm_map_advantage.xlsx')